In [1]:
from sklearn.datasets import fetch_openml

In [2]:
mnist = fetch_openml('mnist_784', version=1,parser ='auto')
X, y = mnist.data, mnist.target

In [3]:
# from ucimlrepo import fetch_ucirepo 

# # fetch dataset 
# optical_recognition_of_handwritten_digits = fetch_ucirepo(id=80) 
  
# # data (as pandas dataframes) 
# X = optical_recognition_of_handwritten_digits.data.features 
# y = optical_recognition_of_handwritten_digits.data.targets 
  
# # metadata 
# print(optical_recognition_of_handwritten_digits.metadata) 

# # variable information 
# print(optical_recognition_of_handwritten_digits.variables) 

In [4]:
# y= y.to_numpy()
# y =y.ravel()

In [3]:
import numpy as np
import random

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [7]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# rf_classifier.fit(X_train, y_train)

# y_pred = rf_classifier.predict(X_test)

# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy}")

In [8]:
# svm_classifier = SVC(kernel='linear', C=1.0)

# svm_classifier.fit(X_train, y_train)

# y_pred = svm_classifier.predict(X_test)

# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy}")

In [9]:
# xgb_classifier = xgb.XGBClassifier()
# xgb_classifier.fit(X_train, y_train)

# y_pred = xgb_classifier.predict(X_test)

# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy}")

In [5]:
def cal_pop_fitness(pop, X_train, y_train, X_test, y_test):
    fitness = []
    for i in range(len(pop)):
        columns = np.unique(pop[i])
        
        X_train_ind = X_train.iloc[:,columns]
        rf_classifier = RandomForestClassifier(n_estimators=5, random_state=42)
        rf_classifier.fit(X_train_ind, y_train)

        y_pred = rf_classifier.predict(X_test.iloc[:,columns])

        accuracy = accuracy_score(y_test, y_pred)

        fitness.append(accuracy)

    return fitness

def fit_svm(pop, X_train, y_train, X_test, y_test):
    fitness = []
    for i in range(len(pop)):
        columns = np.unique(pop[i])
        
        X_train_ind = X_train.iloc[:, columns]
        svm_classifier = SVC(kernel='linear', C=1.0)
        svm_classifier.fit(X_train_ind, y_train)

        y_pred = svm_classifier.predict(X_test.iloc[:, columns])

        accuracy = accuracy_score(y_test, y_pred)

        fitness.append(accuracy)

    return fitness

def fit_xgboost(pop, X_train, y_train, X_test, y_test):
    fitness = []
    for i in range(len(pop)):
        columns = np.unique(pop[i])
        
        X_train_ind = X_train.iloc[:, columns]
        xgb_classifier = xgb.XGBClassifier()
        xgb_classifier.fit(X_train_ind, y_train)

        y_pred = xgb_classifier.predict(X_test.iloc[:, columns])

        accuracy = accuracy_score(y_test, y_pred)

        fitness.append(accuracy)

    return fitness

def select_mating_pool(pop, fitness, num_parents):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = np.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = np.where(fitness == np.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -1
    return parents

def crossover(parents, offspring_size):
    offspring = np.empty(offspring_size)

    for k in range(offspring_size[0]):
        # Always perform one-point crossover.
        crossover_point = np.random.randint(1, parents.shape[1])
        parent1_idx = k % parents.shape[0]
        parent2_idx = (k + 1) % parents.shape[0]
        offspring[k, :crossover_point] = parents[parent1_idx, :crossover_point]
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]

    return offspring

def mutation(offspring_crossover):
    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_crossover.shape[0]):
        # The random value to be added to the gene.
        random_value = np.random.choice([-1, 0, 1], 1)
        offspring_crossover[idx, 4] = offspring_crossover[idx, 4] + random_value
    return offspring_crossover

In [32]:
col_len = 28

sol_per_pop = 32
num_parents_mating = 8

pop_size = (sol_per_pop,col_len)

max_value = 783 
std_dev = 150
mean = max_value / 2

new_population = np.random.normal(mean, std_dev, pop_size).astype(int)

new_population[new_population > max_value] = max_value
new_population[new_population < 0] = 0

print(new_population)

[[109 301 370 292 332 475 118 616 430 333 428 275 483 248 464 346 450 704
  376 254 496 654 248 526 290 410 562 283]
 [443 373 550 727 209 161 569 257 381 204 467 290 520 333 124 623  54 512
  492 503 421 301 346 577 430 270 338 477]
 [399 316 588 451 426 434 208 360 454 447 496 486 408 632 405 390 268 221
  492   0 523 301 288 549 585 591 479 344]
 [392 475 341 407 240 321 197 283 261 471 504 176 260 423 483 684 300 423
  359 187 525 661 466 343  48 574 556 402]
 [536 167 415 319 494 441  95 622 616 420 532 718 430 487 265 416 392 330
  593 335 771 750 160 562 532 339 439 571]
 [272 268 693   0 290 505 348 418 524 439 393 283 418 542 549 528 367 268
  334 282 495 395 479 227 418 619 369 281]
 [379 530 279 373 455 214 110 393 578 430 504 608 292  80 406 448 488 556
  360 550 299 315 452 630 571 651 320 563]
 [577 461 614 343 369 363 273 340 274 418 466 572 233 148 191 222   0 482
  200 415 221 387 524 353 579 334 640 515]
 [266 483 289 271 566 783 587 716 607 460 338 190 593 453 456 53

In [33]:
y=y.astype('int64')

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train = X_train[:10000]
X_test = X_test[:3000]
y_train = y_train[:10000]
y_test = y_test[:3000]

In [35]:
num_generations = 10
for generation in range(num_generations):
    # Measing the fitness of each chromosome in the population.
    fitness = cal_pop_fitness(new_population, X_train,y_train,X_test,y_test)

    print(fitness)

    # Selecting the best parents in the population for mating.
    parents = select_mating_pool(new_population, fitness, 
                                      num_parents_mating)
    
    # print(parents)

    # Generating next generation using crossover.
    offspring_crossover = crossover(parents,
                                       offspring_size=(pop_size[0]-parents.shape[0], col_len))


    # Adding some variations to the offsrping using mutation.
    offspring_mutation = mutation(offspring_crossover)

    # print(offspring_mutation)

    # Creating the new population based on the parents and offspring.
    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation  # should be offspring_mutation

    # The best result in the current iteration.
    print("Generation : ", generation)
    # print(offspring_crossover)
    print()

[0.7366666666666667, 0.8076666666666666, 0.7496666666666667, 0.7556666666666667, 0.6433333333333333, 0.6346666666666667, 0.8083333333333333, 0.722, 0.7356666666666667, 0.7036666666666667, 0.808, 0.8363333333333334, 0.7603333333333333, 0.767, 0.7263333333333334, 0.7166666666666667, 0.7133333333333334, 0.8103333333333333, 0.672, 0.7123333333333334, 0.748, 0.757, 0.738, 0.7516666666666667, 0.775, 0.7246666666666667, 0.7213333333333334, 0.7086666666666667, 0.732, 0.8123333333333334, 0.7136666666666667, 0.6153333333333333]
Generation :  0

[0.8363333333333334, 0.8123333333333334, 0.8103333333333333, 0.8083333333333333, 0.808, 0.8076666666666666, 0.775, 0.767, 0.809, 0.8293333333333334, 0.792, 0.7726666666666666, 0.8363333333333334, 0.8163333333333334, 0.7656666666666667, 0.7856666666666666, 0.8246666666666667, 0.8083333333333333, 0.8013333333333333, 0.7896666666666666, 0.8096666666666666, 0.8096666666666666, 0.7656666666666667, 0.84, 0.8233333333333334, 0.8013333333333333, 0.784, 0.80466666

In [36]:
# Getting the best solution after iterating finishing all generations.
#At first, the fitness is calculated for each solution in the final generation.
fitness = cal_pop_fitness(new_population, X_train, y_train, X_test,y_test)
# Then return the index of that solution corresponding to the best fitness.
best_match_idx = np.where(fitness == np.max(fitness))

# print("Best solution : ", new_population[best_match_idx, :])
print("Best solution fitness : ", fitness[best_match_idx[0][0]])

Best solution fitness :  0.859


In [38]:
print("Best solution : ", new_population[best_match_idx, :])

Best solution :  [[[669 373 550 727 213 161 569 349 270 433 404 148 459 288 346 647 209
   436 256 412 323 443 495 318 236 357 247 486]
  [669 373 550 727 213 161 569 349 270 433 404 148 459 288 346 647 209
   436 256 412 323 443 495 318 236 357 247 486]
  [669 373 550 727 213 161 569 349 270 433 404 148 459 288 346 647 209
   436 256 412 323 443 495 318 236 357 247 486]
  [669 373 550 727 213 161 569 349 270 433 404 148 459 288 346 647 209
   436 256 412 323 443 495 318 236 357 247 486]
  [669 373 550 727 213 161 569 349 270 433 404 148 459 288 346 647 209
   436 256 412 323 443 495 318 236 357 247 486]
  [669 373 550 727 213 161 569 349 270 433 404 148 459 288 346 647 209
   436 256 412 323 443 495 318 236 357 247 486]
  [669 373 550 727 213 161 569 349 270 433 404 148 459 288 346 647 209
   436 256 412 323 443 495 318 236 357 247 486]
  [669 373 550 727 213 161 569 349 270 433 404 148 459 288 346 647 209
   436 256 412 323 443 495 318 236 357 247 486]
  [669 373 550 727 213 161 569 

[233  27 577 438 237 429 197 553 573 487 466 510 348 153 139 612 625
   640 375 342 463 723 656 384 571 380 291 339]

0.8791428571428571

1m 9s

20s

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9667619047619047


In [39]:
feat = new_population[best_match_idx, :]

In [40]:
feat[0][0]

array([669, 373, 550, 727, 213, 161, 569, 349, 270, 433, 404, 148, 459,
       288, 346, 647, 209, 436, 256, 412, 323, 443, 495, 318, 236, 357,
       247, 486])

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train.iloc[:,feat[0][0]], y_train)

y_pred = rf_classifier.predict(X_test.iloc[:,feat[0][0]])
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8866190476190476


In [42]:
new_population = np.random.randint(0, 783, size=28)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train.iloc[:,new_population], y_train)

y_pred = rf_classifier.predict(X_test.iloc[:,new_population])
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.6312380952380953


In [43]:
new_population

array([614, 298, 516, 288, 517, 630,  14, 104,  32, 608, 151, 772, 746,
       170,  15, 575, 645,  71, 671, 124, 127, 499, 490, 704,  86,   7,
       284, 594])

In [31]:
0.9505714285714286/0.96676190476190470 * 100

98.32528814895085